In [26]:
from vanna.ollama import Ollama
from vanna.base import VannaBase
from sentence_transformers import SentenceTransformer


In [78]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Any

class MyCustomVectorDB(VannaBase):
    def __init__(self, config=None):
        super().__init__(config)
        self.ddl_store = []
        self.doc_store = []
        self.question_sql_store = []
        self.id = 0
        self.embedding_model = SentenceTransformer("./Embd_Models/paraphrase-multilingual-MiniLM-L12-v2")  # You'll need to initialize this with your preferred embedding model
        self.config = config
        self.max_tokens = 1024
        self.static_documentation = ""
        self.dialect = ""
        
        
    def add_ddl(self, ddl: str, **kwargs) -> str:
        self.id += 1
        id = str(self.id)
        self.ddl_store.append({"id": id, "ddl": ddl, "embedding": self.generate_embedding(ddl)})
        return id

    def add_documentation(self, doc: str, **kwargs) -> str:
        self.id += 1

        id = str(self.id)
        self.doc_store.append({"id": id, "doc": doc, "embedding": self.generate_embedding(doc)})
        return id

    def add_question_sql(self, question: str, sql: str, **kwargs) -> str:

        self.id += 1
        id = str(self.id)
        self.question_sql_store.append({
            "id": id,
            "question": question,
            "sql": sql,
            "embedding": self.generate_embedding(question)
        })
        return id

    def get_related_ddl(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.ddl_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.ddl_store[i] for i in sorted_indices[:5]]  # Return top 5 related DDLs

    def get_related_documentation(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.doc_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.doc_store[i] for i in sorted_indices[:5]]  # Return top 5 related docs

    def get_similar_question_sql(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.question_sql_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.question_sql_store[i] for i in sorted_indices[:5]]  # Return top 5 similar questions

    def get_training_data(self, **kwargs) -> pd.DataFrame:
        data = []
        for store in [self.ddl_store, self.doc_store, self.question_sql_store]:
            data.extend(store)
        return pd.DataFrame(data)

    def remove_training_data(self, id: str, **kwargs) -> bool:
        for store in [self.ddl_store, self.doc_store, self.question_sql_store]:
            for item in store:
                if item["id"] == id:
                    store.remove(item)
                    return True
        return False

    def generate_embedding(self, text: str, **kwargs) -> np.ndarray:
        
        return self.embedding_model.encode(text)

class MyVanna(MyCustomVectorDB, Ollama):
    def __init__(self, config=None):
        MyCustomVectorDB.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={'model': 'qwen2.5:3b'})

c:\Users\SIVERMA\Documents\Experimenting\NLCHATBOT\NLCHATBOT\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [79]:
# vn.connect_to_sqlite("extracted.db") 
vn.connect_to_sqlite("chinook.db") 

## Training
You only need to train once. Do not train again unless you want to add more training data.

In [80]:

# df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# plan = vn.get_training_plan_generic(df_information_schema)
# plan



#   Incase using sqllite 

df_information_schema = vn.run_sql("SELECT name FROM sqlite_master WHERE type='table';")
df_information_schema



,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


In [81]:
vn.run_sql("select * from artists")

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


In [82]:
# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships

vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS artists (
        ArtistId INT PRIMARY KEY,
        Name VARCHAR(100)
    )
""")

# # Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="""The Chinook sample database has 11 tables as follows:

 employees table stores employee data such as id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
 customers table stores customer data.
 invoices & invoice_items tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
 artists table stores artist data. It is a simple table that contains the id and name.
 albums table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
 media_types table stores media types such as MPEG audio and AAC audio files.
 genres table stores music types such as rock, jazz, metal, etc.
 tracks table stores the data of songs. Each track belongs to one album.
 playlists & playlist_track tables: playlists table stores data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists and tracks tables is many-to-many. The playlist_track table is used to reflect this relationship.""")

# # You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM ArtistId WHERE name = 'Alice In Chains'")


Adding ddl: 
    CREATE TABLE IF NOT EXISTS artists (
        ArtistId INT PRIMARY KEY,
        Name VARCHAR(100)
    )

Adding documentation....
Info: Ollama parameters:
model=qwen2.5:3b,
options={},
keep_alive=None
Info: Prompt Content:
[{"role": "system", "content": "The user will give you SQL and you will try to guess what the business question this query is answering. Return just the question without any additional explanation. Do not reference the table name in the question."}, {"role": "user", "content": "SELECT * FROM ArtistId WHERE name = 'Alice In Chains'"}]
Info: Ollama Response:
{'model': 'qwen2.5:3b', 'created_at': '2024-10-03T12:53:42.2162659Z', 'message': {'role': 'assistant', 'content': 'Which artist named Alice In Chains has been selected?'}, 'done_reason': 'stop', 'done': True, 'total_duration': 1611557900, 'load_duration': 45436700, 'prompt_eval_count': 66, 'prompt_eval_duration': 185276000, 'eval_count': 11, 'eval_duration': 1338398000}
Question generated with sql: 

'3'

In [83]:
len(vn.get_training_data()["embedding"].iloc[0]), vn.get_training_data().shape

(384, (3, 6))

In [84]:
# At any time you can inspect what training data the package is able to reference
vn.get_training_data()

,id,ddl,embedding,doc,question,sql
0,1,\n CREATE TABLE IF NOT EXISTS artists (\n ...,"[0.052741796, 0.09199007, -0.01904367, 0.33062...",NaN,NaN,NaN
1,2,NaN,"[-0.007958114, 0.22179422, -0.27661943, -0.105...",The Chinook sample database has 11 tables as f...,NaN,NaN
2,3,NaN,"[-0.09543198, -0.46606335, 0.025037458, 0.5979...",NaN,Which artist named Alice In Chains has been se...,SELECT * FROM ArtistId WHERE name = 'Alice In ...


In [85]:
# You can remove training data if there's obsolete/incorrect information.
vn.remove_training_data(id='1')


True

## Asking the AI
Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.

In [86]:
vn.ask(question="print top 10 rows from playlist_track")

SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\n{'id': '2', 'doc': 'The Chinook sample database has 11 tables as follows:\\n\\n employees table stores employee data such as id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.\\n customers table stores customer data.\\n invoices & invoice_items tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.\\n artists table stores artist data. It is a simple table that contains the id and name.\\n albums table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.\\n media_types table stores media types such as MPEG audi

# USING OPENAI

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Any
from vanna.openai import OpenAI_Chat
from openai import AzureOpenAI
from vanna.vannadb import VannaDB_VectorStore
from vanna.base import VannaBase
from sentence_transformers import SentenceTransformer



class MyCustomVectorDB(VannaBase):
    def __init__(self, config={}):
        print(config)
        super().__init__(config=config)
        self.ddl_store = []
        self.doc_store = []
        self.question_sql_store = []
        self.id = 0
        self.embedding_model = SentenceTransformer("./Embd_Models/paraphrase-multilingual-MiniLM-L12-v2")  # You'll need to initialize this with your preferred embedding model
        self.config = config
        self.max_tokens = 1024
        self.static_documentation = ""
        self.dialect = ""
        self.temperature = config["temperature"]
        
        
    def add_ddl(self, ddl: str, **kwargs) -> str:
        self.id += 1
        id = str(self.id)
        self.ddl_store.append({"id": id, "ddl": ddl, "embedding": self.generate_embedding(ddl)})
        return id

    def add_documentation(self, doc: str, **kwargs) -> str:
        self.id += 1

        id = str(self.id)
        self.doc_store.append({"id": id, "doc": doc, "embedding": self.generate_embedding(doc)})
        return id

    def add_question_sql(self, question: str, sql: str, **kwargs) -> str:

        self.id += 1
        id = str(self.id)
        self.question_sql_store.append({
            "id": id,
            "question": question,
            "sql": sql,
            "embedding": self.generate_embedding(question)
        })
        return id

    def get_related_ddl(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.ddl_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.ddl_store[i] for i in sorted_indices[:5]]  # Return top 5 related DDLs

    def get_related_documentation(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.doc_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.doc_store[i] for i in sorted_indices[:5]]  # Return top 5 related docs

    def get_similar_question_sql(self, question: str, **kwargs) -> List[Dict[str, Any]]:
        question_embedding = self.generate_embedding(question)
        similarities = [cosine_similarity([question_embedding], [item["embedding"]])[0][0] for item in self.question_sql_store]
        sorted_indices = np.argsort(similarities)[::-1]
        return [self.question_sql_store[i] for i in sorted_indices[:5]]  # Return top 5 similar questions

    def get_training_data(self, **kwargs) -> pd.DataFrame:
        data = []
        for store in [self.ddl_store, self.doc_store, self.question_sql_store]:
            data.extend(store)
        return pd.DataFrame(data)

    def remove_training_data(self, id: str, **kwargs) -> bool:
        for store in [self.ddl_store, self.doc_store, self.question_sql_store]:
            for item in store:
                if item["id"] == id:
                    store.remove(item)
                    return True
        return False

    def generate_embedding(self, text: str, **kwargs) -> np.ndarray:
        
        return self.embedding_model.encode(text)

class MyVanna(MyCustomVectorDB, OpenAI_Chat):
    def __init__(self, config={}):
        OpenAI_Chat.__init__(self, client = AzureOpenAI(
                                                api_key='XXXXXXXXxxxxxxxxxxxxxx',  
                                                api_version="xxxxxxxxxxxxxxxxxx",
                                                azure_endpoint='XXxxxxxxxxxxXXXXXXXXXXXXXXX',
                                            ), config=config)
        MyCustomVectorDB.__init__(self, config=config)
        
        
vn_OAI = MyVanna(config={'temperature':0.1,'model':"xxxxxxxxxxxxxxxxxxxxXXXXXXx"})

c:\Users\SIVERMA\Documents\Experimenting\NLCHATBOT\NLCHATBOT\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


{'temperature': 0.1, 'model': 'NA-MS-OpenAI-gpt-4o-min-Tools'}


c:\Users\SIVERMA\Documents\Experimenting\NLCHATBOT\NLCHATBOT\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# vn.connect_to_sqlite("extracted.db") 
# vn_OAI.connect_to_sqlite("chinook.db") 


# Now we will do postgres

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
vn_OAI.connect_to_postgres(
    host='localhost',
    dbname='Vanna_db',
    user="postgres",
    password="Moodle",
    port=5432
)

In [5]:
df_information_schema = vn_OAI.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn_OAI.get_training_plan_generic(df_information_schema)
plan

Train on Information Schema: Vanna_db.pg_catalog pg_type
Train on Information Schema: Vanna_db.pg_catalog pg_class
Train on Information Schema: Vanna_db.pg_catalog pg_stat_ssl
Train on Information Schema: Vanna_db.pg_catalog pg_index
Train on Information Schema: Vanna_db.pg_catalog pg_attribute
Train on Information Schema: Vanna_db.pg_catalog pg_shseclabel
Train on Information Schema: Vanna_db.pg_catalog pg_indexes
Train on Information Schema: Vanna_db.pg_catalog pg_collation
Train on Information Schema: Vanna_db.pg_catalog pg_database
Train on Information Schema: Vanna_db.pg_catalog pg_amproc
Train on Information Schema: Vanna_db.pg_catalog pg_stat_database
Train on Information Schema: Vanna_db.pg_catalog pg_default_acl
Train on Information Schema: Vanna_db.pg_catalog pg_stat_user_tables
Train on Information Schema: Vanna_db.pg_catalog pg_largeobject
Train on Information Schema: Vanna_db.pg_catalog pg_statio_sys_tables
Train on Information Schema: Vanna_db.pg_catalog pg_shadow
Train o

In [6]:
vn_OAI.train(plan=plan)

In [7]:

vn_OAI.run_sql("select * from actor")


,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620
...,...,...,...,...
195,196,Bela,Walken,2013-05-26 14:47:57.620
196,197,Reese,West,2013-05-26 14:47:57.620
197,198,Mary,Keitel,2013-05-26 14:47:57.620
198,199,Julia,Fawcett,2013-05-26 14:47:57.620


In [8]:

vn_OAI.train(ddl="""
    CREATE TABLE IF NOT EXISTS public.actor
    (
        actor_id integer NOT NULL DEFAULT nextval('actor_actor_id_seq'::regclass),
        first_name character varying(45) COLLATE pg_catalog."default" NOT NULL,
        last_name character varying(45) COLLATE pg_catalog."default" NOT NULL,
        last_update timestamp without time zone NOT NULL DEFAULT now(),
        CONSTRAINT actor_pkey PRIMARY KEY (actor_id)
    )

""")

# # Sometimes you may want to add documentation about your business terminology or definitions.
vn_OAI.train(documentation=
"""
This database includes several tables on invoice information, track, album, artist and 
genre data, and employee and customer information related to the store's sales.

always take tables name with schema
"""
)
# # You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn_OAI.train(
    question="each actor with total number of movies he has done",
    sql="""SELECT actor_id,count(*)
	FROM public.film_actor group by actor_id;"""
    )


Adding ddl: 
    CREATE TABLE IF NOT EXISTS public.actor
    (
        actor_id integer NOT NULL DEFAULT nextval('actor_actor_id_seq'::regclass),
        first_name character varying(45) COLLATE pg_catalog."default" NOT NULL,
        last_name character varying(45) COLLATE pg_catalog."default" NOT NULL,
        last_update timestamp without time zone NOT NULL DEFAULT now(),
        CONSTRAINT actor_pkey PRIMARY KEY (actor_id)
    )


Adding documentation....


'234'

In [9]:
train_df = vn_OAI.get_training_data()
train_df

,id,ddl,embedding,doc,question,sql
0,232,\n CREATE TABLE IF NOT EXISTS public.actor\...,"[-0.09280895, -0.25158352, 0.0012331698, -0.06...",NaN,NaN,NaN
1,1,NaN,"[-0.1558621, -0.29822505, -0.038410503, -0.061...",The following columns are in the pg_type table...,NaN,NaN
2,2,NaN,"[-0.09900052, -0.17877065, -0.11415465, -0.074...",The following columns are in the pg_class tabl...,NaN,NaN
3,3,NaN,"[-0.12304814, -0.20903876, -0.17474012, -0.082...",The following columns are in the pg_stat_ssl t...,NaN,NaN
4,4,NaN,"[-0.11986724, -0.2495471, -0.0042946, -0.07532...",The following columns are in the pg_index tabl...,NaN,NaN
...,...,...,...,...,...,...
229,229,NaN,"[0.020318149, -0.1613569, -0.10487865, -0.1839...",The following columns are in the sales_by_stor...,NaN,NaN
230,230,NaN,"[0.088094294, -0.23672733, -0.06700921, -0.220...",The following columns are in the sales_by_film...,NaN,NaN
231,231,NaN,"[0.077870354, -0.26746625, 0.01768018, -0.2964...",The following columns are in the country table...,NaN,NaN
232,233,NaN,"[0.030890515, -0.14779232, -0.39689556, -0.003...",\nThis database includes several tables on inv...,NaN,NaN


In [12]:
vn_OAI.ask(question="how many movies actor 'Penelope' has done?",allow_llm_to_see_data=True)

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n{\'id\': \'232\', \'ddl\': \'\\n    CREATE TABLE IF NOT EXISTS public.actor\\n    (\\n        actor_id integer NOT NULL DEFAULT nextval(\\\'actor_actor_id_seq\\\'::regclass),\\n        first_name character varying(45) COLLATE pg_catalog."default" NOT NULL,\\n        last_name character varying(45) COLLATE pg_catalog."default" NOT NULL,\\n        last_update timestamp without time zone NOT NULL DEFAULT now(),\\n        CONSTRAINT actor_pkey PRIMARY KEY (actor_id)\\n    )\\n\\n\', \'embedding\': array([-9.28089470e-02, -2.51583517e-01,  1.23316981e-03, -6.50459751e-02,\n       -4.59224917e-02,  2.11858153e-01,  1.61485046e-01,  1.99935004e-01,\n       -3.14955711e-02,  1.04629859e-01,  1.21128701e-01, -1.05873428e-01,\n        2.